### Load Packages

In [229]:
import boto3
import pandas as pd 
import json 
import configparser 
import ndjson
import re 

### Configuration

For security i have made a configuration file that contains my AWS access key and secret key

In [6]:
config=configparser.ConfigParser() 
config.read_file(open('config.cfg'))

KEY = config.get('AWS', 'KEY')
SECRET = config.get('AWS', 'SECRET')

### Connect to the S3 Service

In [8]:
s3 = boto3.resource('s3', 
    region_name = 'us-west-2', 
    aws_access_key_id = KEY, 
    aws_secret_access_key = SECRET
)

### Review the Contents of Bucket

After connecting the service and accessing the bucket lets review the contents of the bucket.  

In [448]:
bucket = s3.Bucket('1up-coding-challenge-patients')

for obj in bucket.objects.all():
    print(obj)

s3.ObjectSummary(bucket_name='1up-coding-challenge-patients', key='AllergyIntolerance.ndjson')
s3.ObjectSummary(bucket_name='1up-coding-challenge-patients', key='CarePlan.ndjson')
s3.ObjectSummary(bucket_name='1up-coding-challenge-patients', key='CareTeam.ndjson')
s3.ObjectSummary(bucket_name='1up-coding-challenge-patients', key='Claim.ndjson')
s3.ObjectSummary(bucket_name='1up-coding-challenge-patients', key='Condition.ndjson')
s3.ObjectSummary(bucket_name='1up-coding-challenge-patients', key='Device.ndjson')
s3.ObjectSummary(bucket_name='1up-coding-challenge-patients', key='DiagnosticReport.ndjson')
s3.ObjectSummary(bucket_name='1up-coding-challenge-patients', key='DocumentReference.ndjson')
s3.ObjectSummary(bucket_name='1up-coding-challenge-patients', key='Encounter.ndjson')
s3.ObjectSummary(bucket_name='1up-coding-challenge-patients', key='ExplanationOfBenefit.ndjson')
s3.ObjectSummary(bucket_name='1up-coding-challenge-patients', key='Group.ndjson')
s3.ObjectSummary(bucket_name='1u

In [412]:
pat = s3.Object('1up-coding-challenge-patients', 'Patient.ndjson')
pat1 = pat.get()['Body'].read()
pat2 = ndjson.loads(pat1.decode('utf-8'))

s3.Object(bucket_name='1up-coding-challenge-patients', key='Patient.ndjson')


In [413]:
pat1 = pat.get()['Body'].read()

In [451]:
pat2 = json.loads(pat1.decode('utf-8'))

JSONDecodeError: Extra data: line 2 column 1 (char 3913)

In [447]:
pat2[1]

{'resourceType': 'Patient',
 'id': 'ebf9231d-6a1f-432a-90c2-bc1b340ae047',
 'meta': {'profile': ['http://standardhealthrecord.org/fhir/StructureDefinition/shr-entity-Patient']},
 'text': {'status': 'generated',
  'div': '<div xmlns="http://www.w3.org/1999/xhtml">Generated by <a href="https://github.com/synthetichealth/synthea">Synthea</a>.Version identifier: v2.5.0-409-g6a2d5846\n .   Person seed: 8987220997374458426  Population seed: 123456789</div>'},
 'extension': [{'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-race',
   'extension': [{'url': 'ombCategory',
     'valueCoding': {'system': 'urn:oid:2.16.840.1.113883.6.238',
      'code': '2106-3',
      'display': 'White'}},
    {'url': 'text', 'valueString': 'White'}]},
  {'url': 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity',
   'extension': [{'url': 'ombCategory',
     'valueCoding': {'system': 'urn:oid:2.16.840.1.113883.6.238',
      'code': '2186-5',
      'display': 'Not Hispanic or Latino'

In [430]:

def get_patient(): 


    pat = s3.Object('1up-coding-challenge-patients', 'Patient.ndjson')
    pat1 = pat.get()['Body'].read()
    pat2 = ndjson.loads(pat1.decode('utf-8'))

    h = input("Enter First Name: ")
    v = input("Enter last name: ")

    fnl, lnl, idl = None, None, None

    for name in pat2: 

        if name['name'][0]['family'] == v and name['name'][0]['given'][0] == h: 

            fnl = name['name'][0]['given'][0]
            lnl = name['name'][0]['family']
            idl = name['id']    
    
    return fnl, lnl, idl 




In [431]:
fn, ln, id = get_patient()

In [439]:
get_patient()(3)

TypeError: 'tuple' object is not callable

In [432]:
print(fn, ln, id)

Cleo27 Bode78 d13874ec-22ea-46ed-a55c-1fd75ef56a58


'Bode78 Cleo27'

In [151]:
if a['name'][0]['family'] == 'Bode78' and a['name'][0]['given'][0] == 'Cleo27': 
    print('YES')

else: 
    print('No')

YES


Need to ndjson library to be able to read ndjson files 
https://pypi.org/project/ndjson/


In [92]:
buckets = list(bucket.objects.all())

In [174]:
content3 = s3.Object('1up-coding-challenge-patients', 'Observation.ndjson')
print(content3)

s3.Object(bucket_name='1up-coding-challenge-patients', key='Observation.ndjson')


In [214]:
content4 = content3.get()['Body'].read()
json_data1 = ndjson.loads(content4.decode('utf-8'))


In [212]:
idl1 = 'Patient/'+idl


In [241]:
count = {'a' : 0}
for item in json_data1: 
    if idl in item['subject']['reference']: 
        count['a'] += 1

In [242]:
count

{'a': 93}

In [323]:
d = {}
ls = []
for obj in bucket.objects.all(): 
    filename = []
    filename.append(obj.key)

    if obj.key != 'Patient.ndjson': 
        hl = obj.key

        content = obj

        content1 = content.get()['Body'].read()
        json_data = ndjson.loads(content1.decode('utf-8'))

        for item in filename: 
            bar = re.sub(".ndjson", '', str(item))
            new = bar
            for item in json_data: 
                if item[0]['resourceType'] == new:
                    if idl in item[0]['target'][0]: 
                        ls.append(new)

                for i in item: 
                    if i == 'subject': 
                        if idl in item['subject']['reference']: 
                            ls.append(new)
                    elif i == 'patient': 
                        if idl in item['patient']['reference']: 
                            ls.append(new)
                    elif i == 'target': 
                        if idl in item['target'][:]: 
                            ls.append(new)




KeyError: 0

In [388]:
a = Counter(ls).keys()
b = Counter(ls).values()
print(a)
print(b)

In [389]:
print(a)
print(b)

dict_keys(['CarePlan', 'CareTeam', 'Claim', 'Condition', 'DiagnosticReport', 'DocumentReference', 'Encounter', 'ExplanationOfBenefit', 'Immunization', 'Observation', 'Procedure', 'Provenance'])
dict_values([2, 2, 8, 8, 12, 8, 8, 8, 11, 93, 5, 1])


In [401]:
ls1 = []
for obj in bucket.objects.all(): 
    filename = []
    filename.append(obj.key)

    if obj.key != 'Patient.ndjson': 
        hl = obj.key

        content = obj

        content1 = content.get()['Body'].read()
        json_data = ndjson.loads(content1.decode('utf-8'))
        ls1.append(json_data)

json_data[0]

{'resourceType': 'SupplyDelivery',
 'id': 'afe96e02-30f7-4623-b3f4-0b723d1c5815',
 'status': 'completed',
 'patient': {'reference': 'Patient/421c3eaf-f95c-47af-b8cd-f6cbcb192fad'},
 'type': {'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/supply-item-type',
    'code': 'device',
    'display': 'Device'}]},
 'suppliedItem': {'quantity': {'value': 2},
  'itemCodeableConcept': {'coding': [{'system': 'http://snomed.info/sct',
     'code': '409534002',
     'display': 'Disposable air-purifying respirator (physical object)'}],
   'text': 'Disposable air-purifying respirator (physical object)'}},
 'occurrenceDateTime': '2020-03-13T20:45:25-04:00'}

In [222]:
import re 

newlist = [] 

for item in filename: 
    bar = re.sub(".ndjson", '', str(item))
    newlist.append(bar)

In [427]:
for item in ls1: 
    if item[0]['resourceType'] == 'Location':
        #print(item[0]['target'][0])
        a = item

In [428]:
a[0]

{'resourceType': 'Location',
 'id': '2b38f05a-19f4-4bb5-b0f7-3a7a7e3a8f28',
 'meta': {'profile': ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-location']},
 'status': 'active',
 'name': 'LAHEY HOSPITAL & MEDICAL CENTER, BURLINGTON',
 'telecom': [{'system': 'phone', 'value': '7817445100'}],
 'address': {'line': ['41 & 45 MALL ROAD'],
  'city': 'BURLINGTON',
  'state': 'MA',
  'postalCode': '01803',
  'country': 'US'},
 'position': {'longitude': -71.201713, 'latitude': 42.503227},
 'managingOrganization': {'reference': 'Organization/6f122869-a856-3d65-8db9-099bf4f5bbb8',
  'display': 'LAHEY HOSPITAL & MEDICAL CENTER, BURLINGTON'}}

In [435]:

def pat_info(): 

    ls = []
    for obj in bucket.objects.all(): 
        filename = []
        filename.append(obj.key)

        if obj.key != 'Patient.ndjson': 
            hl = obj.key

            content = obj

            content1 = content.get()['Body'].read()
            json_data = ndjson.loads(content1.decode('utf-8'))

            for item in filename: 
                bar = re.sub(".ndjson", '', str(item))
                new = bar
                for item in json_data: 
                    if item['resourceType'] == new:
                        if 'patient' in item: 
                            if id in item['patient']['reference']: 
                                ls.append(new)
                        elif 'subject' in item: 
                            if id in item['subject']['reference']:
                                ls.append(new)
                        elif 'target' in item: 
                            for x in item['target']: 
                                if id in x['reference']:
                                    ls.append(new)
                            #if idl in item['target'][0]:
                                #ls1.append(new)
                        #print(item)
                        #if idl in item[0]['target'][0]: 
                            #ls.append(new)
    a = Counter(ls).keys()
    b = Counter(ls).values()

    final = pd.DataFrame(list(zip(a,b)), columns = ['RESOURCE_TYPE', 'COUNT'])
    return final.sort_values(['COUNT'], ascending=False)
    

In [436]:
pat_info()

,RESOURCE_TYPE,COUNT
9,Observation,93
4,DiagnosticReport,12
8,Immunization,11
2,Claim,8
3,Condition,8
5,DocumentReference,8
6,Encounter,8
7,ExplanationOfBenefit,8
10,Procedure,5
0,CarePlan,2


In [434]:
a = Counter(ls).keys()
b = Counter(ls).values()
print(a)
print(b)

dict_keys(['CarePlan', 'CareTeam', 'Claim', 'Condition', 'DiagnosticReport', 'DocumentReference', 'Encounter', 'ExplanationOfBenefit', 'Immunization', 'Observation', 'Procedure', 'Provenance'])
dict_values([2, 2, 8, 8, 12, 8, 8, 8, 11, 93, 5, 1])


In [422]:
list(zip(a, b))

[('CarePlan', 2),
 ('CareTeam', 2),
 ('Claim', 8),
 ('Condition', 8),
 ('DiagnosticReport', 12),
 ('DocumentReference', 8),
 ('Encounter', 8),
 ('ExplanationOfBenefit', 8),
 ('Immunization', 11),
 ('Observation', 93),
 ('Procedure', 5),
 ('Provenance', 1)]

In [426]:
final = pd.DataFrame(list(zip(a,b)), columns = ['RESOURCE_TYPE', 'COUNT'])
final.sort_values(['COUNT'], ascending=False)

,RESOURCE_TYPE,COUNT
9,Observation,93
4,DiagnosticReport,12
8,Immunization,11
2,Claim,8
3,Condition,8
5,DocumentReference,8
6,Encounter,8
7,ExplanationOfBenefit,8
10,Procedure,5
0,CarePlan,2


In [441]:
id

'd13874ec-22ea-46ed-a55c-1fd75ef56a58'

In [444]:
 
patient_id = 'd13874ec-22ea-46ed-a55c-1fd75ef56a58'

def get_patient(id): 




    pat = s3.Object('1up-coding-challenge-patients', 'Patient.ndjson')
    pat1 = pat.get()['Body'].read()
    pat2 = ndjson.loads(pat1.decode('utf-8'))


    fnl, lnl, idl = None, None, None

    for name in pat2: 

        if name['id']==patient_id: 

            fnl = name['name'][0]['given'][0]
            lnl = name['name'][0]['family']
            idl = name['id']    
    
    return fnl, lnl, idl 

In [445]:
fn, ln, id = get_patient(patient_id)

In [446]:
fn

'Cleo27'